In [131]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import StratifiedKFold

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [134]:
from common.options import set_options
from data.get_data import get_X_y_days, get_X_y_S1_S2

options = set_options()
options['multilabel'] = False
options['task'] = 'DualGo'

X_days, y_days = get_X_y_days(IF_PREP=1)
print(y_days.head())
X_S1_S2, y_S1_S2 = get_X_y_S1_S2(X_days, y_days, **options)

print('X', X_S1_S2.shape, 'y', y_S1_S2.shape)

loading files from /home/leon/bebopalula/python/dual/data/AP12


avg noise over trials


   sample_odor  dist_odor     tasks   response  laser  day
0          1.0        0.0       DPA  incorrect    0.0  1.0
1          0.0        0.0       0.0    correct    0.0  1.0
2          3.0        4.0  DualNoGo  incorrect    0.0  1.0
3          0.0        3.0  DualNoGo  incorrect    0.0  1.0
4          2.0        1.0    DualGo    correct    0.0  1.0
features sample tasks DualGo trials  days last laser 0
X_S1 (64, 741, 115) X_S2 (64, 741, 115)
X_S3 (0, 741, 115) X_S4 (0, 741, 115)
X (128, 741, 115) y (128,)


In [210]:
def array_to_ts(data, batch, length):

  out = tf.keras.utils.timeseries_dataset_from_array(
    data= data[:-10],    
    targets= data[10:],    
    sequence_length=length,
    sequence_stride=1,
    shuffle=False,
    batch_size=batch-1)
  
  return out
  

In [213]:
def prep_df(df):

  print('df', df.shape)
  batch = df.shape[0]
  neurons = df.shape[1]
  length = df.shape[-1]
  
  print('batch', batch, 'length', length)
  X = np.vstack(np.swapaxes(df, 1, 2))
  print('X', X.shape)
  tf_X = array_to_ts(X, batch, length)

  return tf_X
  

In [214]:
n = X_S1_S2.shape[-1]
print(n)
df_train = X_S1_S2[0:int(n*0.7)]
df_val = X_S1_S2[int(n*0.7):int(n*0.9)]

tf_train = prep_df(df_train)
tf_val = prep_df(df_val)

print(tf_train.element_spec)
print(tf_val.element_spec)


115
df (80, 741, 115)
batch 80 length 115
X (9200, 741)
df (23, 741, 115)
batch 23 length 115
X (2645, 741)


(TensorSpec(shape=(None, None, 741), dtype=tf.float64, name=None), TensorSpec(shape=(None, 741), dtype=tf.float64, name=None))
(TensorSpec(shape=(None, None, 741), dtype=tf.float64, name=None), TensorSpec(shape=(None, 741), dtype=tf.float64, name=None))


In [254]:
tf_train = tf.data.Dataset.from_tensors(np.swapaxes(df_train[..., :-10], 1, 2))
tf_val = tf.data.Dataset.from_tensors(np.swapaxes(df_val[..., :-10], 1, 2))
print(tf_train)
print(tf_val)

y_train = tf.data.Dataset.from_tensors(np.swapaxes(df_train[..., 10:], 1, 2))
y_val = tf.data.Dataset.from_tensors(np.swapaxes(df_val[...,10:], 1, 2))


<TensorDataset element_spec=TensorSpec(shape=(80, 115, 741), dtype=tf.float64, name=None)>
<TensorDataset element_spec=TensorSpec(shape=(23, 115, 741), dtype=tf.float64, name=None)>


In [238]:
df_train.shape

(80, 741, 115)

In [245]:
from keras.preprocessing.sequence import TimeseriesGenerator

train = np.hstack(np.vstack(df_train.T))
val = np.hstack(np.vstack(df_val.T))

gen_train = TimeseriesGenerator(train, train, length=115, batch_size=80)
gen_val = TimeseriesGenerator(val, val, length=115, batch_size=80)


In [225]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2))

print(dataset.shape)

(10, 2)


In [255]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=741, return_sequences= True, input_shape=(115, neurons)))
model.add(tf.keras.layers.Dense(units=741))
model.summary()

Model: "sequential_23"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


 lstm_22 (LSTM)              (None, 115, 741)          4395612   


 dense_22 (Dense)            (None, 115, 741)          549822    


Total params: 4,945,434


Trainable params: 4,945,434


Non-trainable params: 0


_________________________________________________________________


In [256]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [257]:
model.fit(tf_train, validation_data=tf_val)

ValueError: in user code:

    File "/home/leon/mambaforge/lib/python3.8/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/leon/mambaforge/lib/python3.8/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/leon/mambaforge/lib/python3.8/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/leon/mambaforge/lib/python3.8/site-packages/keras/engine/training.py", line 1025, in train_step
        self._validate_target_and_loss(y, loss)
    File "/home/leon/mambaforge/lib/python3.8/site-packages/keras/engine/training.py", line 979, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.MeanSquaredError object at 0x7fa6877d1a90>, and therefore expe

In [ ]:
    # Store the raw data.
    print('df', df.shape)

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift
    
    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    n = df.shape[0]
    self.train = tf.convert_to_tensor(df[0:int(n*0.7)])
    self.val = tf.convert_to_tensor(df[int(n*0.7):int(n*0.9)])
    self.test = tf.convert_to_tensor(df[int(n*0.9):])

    return self
  